In [ ]:
#System Libraries
from datetime import datetime
from pandas.tseries.offsets import BDay

#DataFrames manipulation
import pandas as pd

#Import the Normal Distribution from scipy package
from scipy.stats import norm

#Data download from Yahoo Finance
from yahooquery import Ticker
from fake_useragent import UserAgent

#Download Tables from websites
import requests

#Track the progress of the loop
from tqdm import tqdm 

#Libraries for the Plotting
import holoviews as hv
from holoviews import opts, dim
from holoviews.plotting.links import RangeToolLink
from bokeh.models import HoverTool

#Librarie to save the plots to html object
import panel as pn
import param

hv.extension('bokeh')

# Russell 3000 Index - Download the Financial Data and Calculate the M-Score

In [ ]:
#Import the Russell 3000 Index (RAY) based on IWV ETF
ray_df = pd.read_csv('IWV_holdings_12302022_clean.csv')

#Filter for the Stocks - Delete Cash, Futures and so on
ray_df = ray_df[ray_df['Asset Class'] == 'Equity']

#Filter for the Stocks which are NOT in the Financials sector
ray_df_ex_financials = ray_df[ray_df['Sector'] != 'Financials']

#Get the Ticker list
ticker_list = ray_df_ex_financials['Ticker'].to_list()

In [ ]:
#Create an empty DataFrame where we will Concatenate the information
final_df_full = pd.DataFrame(columns=['Ticker', 'Name', 'Beneish M-Score','Probability of Manipulation (%)'])

In [ ]:
#Run the loop for all companies in Ticker List
for ticker in tqdm(ticker_list):
    try:
        #Instantiate the Yahoo Finance and UserAgent module
        ua = UserAgent()
        stock = Ticker(ticker, user_agent=ua.random)

        #Download the Balance Sheet and Income Statement from Yahoo Finance
        balance_sheet_df = stock.balance_sheet(frequency='a')
        income_statement_df = stock.income_statement(frequency='a', trailing=False)
        cashflow_statement_df = stock.cash_flow(frequency='a', trailing=False)

        #Limit the Data for the Last Period and 1 Year Before of the Financial Statements 
        balance_sheet_df_t0 = pd.DataFrame(balance_sheet_df.iloc[-1]).T
        balance_sheet_df_tm1 = pd.DataFrame(balance_sheet_df.iloc[-2]).T

        income_statement_df_t0 = pd.DataFrame(income_statement_df.iloc[-1]).T
        income_statement_df_tm1 = pd.DataFrame(income_statement_df.iloc[-2]).T

        cashflow_statement_df_t0 = pd.DataFrame(cashflow_statement_df.iloc[-1]).T
        cashflow_statement_df_tm1 = pd.DataFrame(cashflow_statement_df.iloc[-2]).T

        #Obtaining the Financial Variables and Ratios to be used in the Beneish Model
        receivables_t0 = balance_sheet_df_t0['AccountsReceivable'][0]
        receivables_tm1 = balance_sheet_df_tm1['AccountsReceivable'][0]

        sales_t0 = income_statement_df_t0['TotalRevenue'][0]
        sales_tm1 = income_statement_df_tm1['TotalRevenue'][0]

        gross_margin_t0 = income_statement_df_t0['GrossProfit'][0] / sales_t0
        gross_margin_tm1 = income_statement_df_tm1['GrossProfit'][0] / sales_tm1

        ppe_t0 = balance_sheet_df_t0['GrossPPE'][0]
        ppe_tm1 = balance_sheet_df_tm1['GrossPPE'][0]

        current_assets_t0 = balance_sheet_df_t0['CurrentAssets'][0]
        current_assets_tm1 = balance_sheet_df_tm1['CurrentAssets'][0]

        total_assets_t0 = balance_sheet_df_t0['TotalAssets'][0]
        total_assets_tm1 = balance_sheet_df_tm1['TotalAssets'][0]

        depreciation_t0 = income_statement_df_t0['ReconciledDepreciation'][0]
        depreciation_tm1 = income_statement_df_tm1['ReconciledDepreciation'][0]

        depreciation_rate_t0 = depreciation_t0 / (depreciation_t0 + ppe_t0)
        depreciation_rate_tm1 = depreciation_tm1 / (depreciation_tm1 + ppe_tm1)

        sga_t0 = income_statement_df_t0['OperatingExpense'][0] if income_statement_df_t0.get('SellingGeneralAndAdministration') is None else income_statement_df_t0['SellingGeneralAndAdministration'][0]
        sga_tm1 = income_statement_df_tm1['OperatingExpense'][0] if income_statement_df_tm1.get('SellingGeneralAndAdministration') is None else income_statement_df_tm1['SellingGeneralAndAdministration'][0]

        net_income_t0 = cashflow_statement_df_t0['NetIncome'][0]
        operating_cf_t0 = cashflow_statement_df_t0['OperatingCashFlow'][0]

        total_debt_t0 = balance_sheet_df_t0['TotalDebt'][0]
        total_debt_tm1 = balance_sheet_df_t0['TotalDebt'][0]

        leverage_t0 = total_debt_t0/total_assets_t0
        leverage_tm1 = total_debt_tm1/total_assets_tm1

        #The Beneish Model is comprised of 8 ratios -> Calculated below
        dsr = (receivables_t0/sales_t0) / (receivables_tm1/sales_tm1)
        gmi = gross_margin_tm1/gross_margin_t0
        aqi = (1 - (ppe_t0 + current_assets_t0)/total_assets_t0) / (1 - (ppe_tm1 + current_assets_tm1)/total_assets_tm1)
        sgi = sales_t0/sales_tm1
        depi = depreciation_rate_tm1/depreciation_rate_t0
        sgai = (sga_t0/sales_t0) / (sga_tm1/sales_tm1)
        accruals = (net_income_t0 - operating_cf_t0) / total_assets_t0
        levi = leverage_t0/leverage_tm1

        #Plugging all the values in the model
        beneish_mscore = round((-4.84 + 0.920 * dsr + 0.528 * gmi + 0.404 * aqi + 0.892 * sgi + 0.115 * depi - 0.172 * sgai 
                          + 4.679 * accruals - 0.327 * levi), 2)

        #Calculate the Probability of Earnings manipulation based on the Normal Distribution
        probability_maniputalition = round((norm.cdf(beneish_mscore) * 100), 2)


        #Create a DataFrame with the Final Data for each Stock
        name = ray_df[ray_df['Ticker'] == ticker]['Name'].iloc[0]
        final_df = pd.DataFrame([{'Ticker':ticker, 'Name':name, 'Beneish M-Score':beneish_mscore, 'Probability of Manipulation (%)':probability_maniputalition}])

        #Concatenate current Ticker's Data to the Full DataFrame
        final_df_full = pd.concat([final_df_full, final_df], axis=0)
        
    except:
        next

#Add a Column with Sector
final_df_full = final_df_full.merge(ray_df[['Ticker','Sector']], on='Ticker', how='left')

#Reorder column order
final_df_full = final_df_full[['Ticker', 'Name', 'Sector', 'Beneish M-Score', 'Probability of Manipulation (%)']]

#Remove the NaNs
final_df_full = final_df_full.dropna()

In [ ]:
#Create Holoviews table object for All Companies
full_df_table = hv.Table(final_df_full).opts(
    opts.Table(width=950, height=280, selectable = True, index_position = None, 
               title = 'Beneish M-Score for All Companies'))

#Save the Plots
p = pn.panel(full_df_table)
p.save('Beneish_MScore_All_Companies.html', embed = True)

# Create the Table with the Top 10 Largest Companies (ex-Financials)

In [ ]:
#Create a Dataframe with Only the Top 10
top_10_df = final_df_full[0:10]

In [ ]:
#Create Holoviews table object for the Top 10 Table
top_10_table = hv.Table(top_10_df).opts(
    opts.Table(width=950, height=280, selectable = True, index_position = None, 
               title = 'Beneish M-Score for Top 10 Largest Companies by Market Cap (ex-Financials)'))

#Save the Plots
p = pn.panel(top_10_table)
p.save('Beneish_MScore_Top10_Companies.html', embed = True)

# Data for the List of Companies which filled for Bankruptcy so far in 2023

In [ ]:
#Get the table with the data for the Bankruptcies that already occured in 2023 from StockAnalysis website
ua = UserAgent()
header = {"User-Agent": ua.random}

r = requests.get('https://stockanalysis.com/actions/bankruptcies/2023/', headers = header)

bankrupcies_table = pd.read_html(r.content)[0]

bankrupcies_ticker_list = bankrupcies_table['Symbol'].to_list()

In [ ]:
#Create an empty DataFrame where we will Concatenate the information
bankruptcies_df_full = pd.DataFrame(columns=['Ticker', 'Name', 'Beneish M-Score','Probability of Manipulation (%)'])

#Run the Loop for the Bankrupt Companies
for ticker in tqdm(bankrupcies_ticker_list):
    try:
        #Instantiate the Yahoo Finance and UserAgent module
        ua = UserAgent()
        stock = Ticker(ticker, user_agent=ua.random)

        #Download the Balance Sheet and Income Statement from Yahoo Finance
        balance_sheet_df = stock.balance_sheet(frequency='a')
        income_statement_df = stock.income_statement(frequency='a', trailing=False)
        cashflow_statement_df = stock.cash_flow(frequency='a', trailing=False)

        #Limit the Data for the Last Period and 1 Year Before of the Financial Statements 
        balance_sheet_df_t0 = pd.DataFrame(balance_sheet_df.iloc[-1]).T
        balance_sheet_df_tm1 = pd.DataFrame(balance_sheet_df.iloc[-2]).T

        income_statement_df_t0 = pd.DataFrame(income_statement_df.iloc[-1]).T
        income_statement_df_tm1 = pd.DataFrame(income_statement_df.iloc[-2]).T

        cashflow_statement_df_t0 = pd.DataFrame(cashflow_statement_df.iloc[-1]).T
        cashflow_statement_df_tm1 = pd.DataFrame(cashflow_statement_df.iloc[-2]).T

        #Obtaining the Financial Variables and Ratios to be used in the Beneish Model
        receivables_t0 = balance_sheet_df_t0['AccountsReceivable'][0]
        receivables_tm1 = balance_sheet_df_tm1['AccountsReceivable'][0]

        sales_t0 = income_statement_df_t0['TotalRevenue'][0]
        sales_tm1 = income_statement_df_tm1['TotalRevenue'][0]

        gross_margin_t0 = income_statement_df_t0['GrossProfit'][0] / sales_t0
        gross_margin_tm1 = income_statement_df_tm1['GrossProfit'][0] / sales_tm1

        ppe_t0 = balance_sheet_df_t0['GrossPPE'][0]
        ppe_tm1 = balance_sheet_df_tm1['GrossPPE'][0]

        current_assets_t0 = balance_sheet_df_t0['CurrentAssets'][0]
        current_assets_tm1 = balance_sheet_df_tm1['CurrentAssets'][0]

        total_assets_t0 = balance_sheet_df_t0['TotalAssets'][0]
        total_assets_tm1 = balance_sheet_df_tm1['TotalAssets'][0]

        depreciation_t0 = income_statement_df_t0['ReconciledDepreciation'][0]
        depreciation_tm1 = income_statement_df_tm1['ReconciledDepreciation'][0]

        depreciation_rate_t0 = depreciation_t0 / (depreciation_t0 + ppe_t0)
        depreciation_rate_tm1 = depreciation_tm1 / (depreciation_tm1 + ppe_tm1)

        sga_t0 = income_statement_df_t0['OperatingExpense'][0] if income_statement_df_t0.get('SellingGeneralAndAdministration') is None else income_statement_df_t0['SellingGeneralAndAdministration'][0]
        sga_tm1 = income_statement_df_tm1['OperatingExpense'][0] if income_statement_df_tm1.get('SellingGeneralAndAdministration') is None else income_statement_df_tm1['SellingGeneralAndAdministration'][0]

        net_income_t0 = cashflow_statement_df_t0['NetIncome'][0]
        operating_cf_t0 = cashflow_statement_df_t0['OperatingCashFlow'][0]

        total_debt_t0 = balance_sheet_df_t0['TotalDebt'][0]
        total_debt_tm1 = balance_sheet_df_t0['TotalDebt'][0]

        leverage_t0 = total_debt_t0/total_assets_t0
        leverage_tm1 = total_debt_tm1/total_assets_tm1

        #The Beneish Model is comprised of 8 ratios -> Calculated below
        dsr = (receivables_t0/sales_t0) / (receivables_tm1/sales_tm1)
        gmi = gross_margin_tm1/gross_margin_t0
        aqi = (1 - (ppe_t0 + current_assets_t0)/total_assets_t0) / (1 - (ppe_tm1 + current_assets_tm1)/total_assets_tm1)
        sgi = sales_t0/sales_tm1
        depi = depreciation_rate_tm1/depreciation_rate_t0
        sgai = (sga_t0/sales_t0) / (sga_tm1/sales_tm1)
        accruals = (net_income_t0 - operating_cf_t0) / total_assets_t0
        levi = leverage_t0/leverage_tm1

        #Plugging all the values in the model
        beneish_mscore = round((-4.84 + 0.920 * dsr + 0.528 * gmi + 0.404 * aqi + 0.892 * sgi + 0.115 * depi - 0.172 * sgai 
                          + 4.679 * accruals - 0.327 * levi), 2)

        #Calculate the Probability of Earnings manipulation based on the Normal Distribution
        probability_maniputalition = round((norm.cdf(beneish_mscore) * 100), 2)


        #Create a DataFrame with the Final Data for each Stock
        name = bankrupcies_table[bankrupcies_table['Symbol'] == ticker]['Company Name'].iloc[0]
        final_df = pd.DataFrame([{'Ticker':ticker, 'Name':name, 'Beneish M-Score':beneish_mscore, 'Probability of Manipulation (%)':probability_maniputalition}])

        #Concatenate current Ticker's Data to the Full DataFrame
        bankruptcies_df_full = pd.concat([bankruptcies_df_full, final_df], axis=0)
    
    except:
        next

#Remove the NaNs
bankruptcies_df_full = bankruptcies_df_full.dropna()

In [ ]:
#Create Holoviews table object for the Bankruptices Table
bankruptcies_table = hv.Table(bankruptcies_df_full).opts(
    opts.Table(width=950, height=280, selectable = True, index_position = None, 
               title = 'Beneish M-Score for Companies which Filled for Bankruptcy in 2023'))

#Save the Plots
p = pn.panel(bankruptcies_table)
p.save('Beneish_MScore_Bankrupt_Companies.html', embed = True)

# Create the Table with the Companies with High Probability of Earnings Manipulation

In [ ]:
#Filter the Table to get the Companies with a Probability of Manipulating Earnings above 3.8%
high_prob_manipulation_df = final_df_full[final_df_full['Probability of Manipulation (%)'] > 3.8]

In [ ]:
#Create Holoviews table object for the Top 10 Table
high_prob_manipulation_table = hv.Table(high_prob_manipulation_df).opts(
    opts.Table(width=950, height=280, selectable = True, index_position = None, 
               title = 'Beneish M-Score for Companies with Probability of Earnings Manipulation above 3.8%'))

#Save the Plots
p = pn.panel(high_prob_manipulation_table)
p.save('Beneish_MScore_High_Prob_Manipulation_Companies.html', embed = True)